In [1]:
import os
import urllib.request
import pandas as pd

In [2]:
dataFol = os.getcwd() + "/dataFiles/"
nysHealth = dataFol + "nysHealth/"

In [3]:
# Hospital Inpatient Discharges (SPARCS De-Identified)
# from https://health.data.ny.gov/Health/Hospital-Inpatient-Discharges-SPARCS-De-Identified/gnzp-ekau
# The Statewide Planning and Research Cooperative System (SPARCS) Inpatient De-identified File contains discharge
# level detail on patient characteristics, diagnoses, treatments, services, and charges. This data file contains 
# basic record level detail for the discharge. The de-identified data file does not contain data that is protected 
# health information (PHI) under HIPAA. The health information is not individually identifiable; all data elements 
# considered identifiable have been redacted. For example, the direct identifiers regarding a date have the day and 
# month portion of the date removed.

sparcsUrl = ["https://health.data.ny.gov/api/views/gnzp-ekau/rows.csv?accessType=DOWNLOAD", # sparcs 2016
             "https://health.data.ny.gov/api/views/82xm-y6g8/rows.csv?accessType=DOWNLOAD", # sparcs 2015
             "https://health.data.ny.gov/api/views/rmwa-zns4/rows.csv?accessType=DOWNLOAD", # sparcs 2014
             "https://health.data.ny.gov/api/views/npsr-cm47/rows.csv?accessType=DOWNLOAD"] # sparcs 2013

In [4]:
sparcsFiles = ["sparcs2016.csv", 
               "sparcs2015.csv",
               "sparcs2014.csv",
               "sparcs2013.csv"]

In [5]:
if not os.path.isdir(nysHealth):
    os.system('mkdir '+ nysHealth)
else:
    print('you can continue')

you can continue


In [6]:
for sp in range(len(sparcsFiles)):
    ### check that the file is not already there
    if not os.path.isfile(nysHealth + sparcsFiles[sp]):
        if os.path.isfile(sparcsFiles[sp]):
            # if in the current dir just move it
            os.system("mv " + sparcsFiles[sp] + " " + nysHealth)
            print ("Moving file!, Please check!")
        #otherwise start looking for the zip file
        else:
            if not os.path.isfile(sparcsFiles[sp]):
                urllib.request.urlretrieve(sparcsUrl[sp], sparcsFiles[sp])
                ###  To move it I use the os.system() functions to run bash commands with arguments
                os.system("mv " + sparcsFiles[sp] + " " + nysHealth)
    ### One final check:
    if not os.path.isfile(nysHealth + sparcsFiles[sp]):
        print ("WARNING!!! something is wrong: the file is not there!")
    else:
        print ("file in place, you can continue")

file in place, you can continue
file in place, you can continue
file in place, you can continue
file in place, you can continue


In [7]:
collect_df = {}

for i in range(len(sparcsFiles)):
    df_name = sparcsFiles[i].split(".")[:-1][0]
    temp = pd.read_csv(nysHealth + sparcsFiles[i])
    temp['yr'] = sparcsFiles[i].split(".")[:-1][0][6:]
    temp.rename(columns={'Facility ID':'Facility Id'}, inplace=True)
    print(temp.shape)
    collect_df[df_name] = temp

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(2343429, 39)
(2346931, 38)
(2367550, 38)
(2428905, 38)


In [8]:
sparcsAll = pd.concat(collect_df, ignore_index=True, keys=None)#, sort=False)

In [9]:
sparcsAll.shape

(9486815, 39)

In [10]:
#cleaning to only knee related
sparcsKnee = sparcsAll[sparcsAll['APR DRG Code']==302]
sparcsKnee.head()

,APR DRG Code,APR DRG Description,APR MDC Code,APR MDC Description,APR Medical Surgical Description,APR Risk of Mortality,APR Severity of Illness Code,APR Severity of Illness Description,Abortion Edit Indicator,Age Group,...,Payment Typology 1,Payment Typology 2,Payment Typology 3,Race,Ratio of Total Costs to Total Charges,Total Charges,Total Costs,Type of Admission,Zip Code - 3 digits,yr
1822,302,Knee joint replacement,8,Diseases and Disorders of the Musculoskeletal ...,Surgical,Minor,2,Moderate,N,70 or Older,...,Blue Cross/Blue Shield,Medicare,NaN,White,NaN,25128.07,49633.70,Elective,147,2013
1883,302,Knee joint replacement,8,Diseases and Disorders of the Musculoskeletal ...,Surgical,Minor,1,Minor,N,50 to 69,...,Blue Cross/Blue Shield,NaN,NaN,White,NaN,34503.42,84472.01,Elective,147,2013
1884,302,Knee joint replacement,8,Diseases and Disorders of the Musculoskeletal ...,Surgical,Minor,2,Moderate,N,50 to 69,...,Medicaid,NaN,NaN,White,NaN,54106.70,94418.08,Elective,148,2013
1888,302,Knee joint replacement,8,Diseases and Disorders of the Musculoskeletal ...,Surgical,Moderate,2,Moderate,N,50 to 69,...,Blue Cross/Blue Shield,Medicare,NaN,White,NaN,30138.52,51083.21,Elective,OOS,2013
1898,302,Knee joint replacement,8,Diseases and Disorders of the Musculoskeletal ...,Surgical,Major,2,Moderate,N,70 or Older,...,Medicare,NaN,NaN,White,NaN,25886.07,49957.54,Elective,147,2013


In [11]:
sparcsKnee.columns

Index(['APR DRG Code', 'APR DRG Description', 'APR MDC Code',
       'APR MDC Description', 'APR Medical Surgical Description',
       'APR Risk of Mortality', 'APR Severity of Illness Code',
       'APR Severity of Illness Description', 'Abortion Edit Indicator',
       'Age Group', 'Attending Provider License Number', 'Birth Weight',
       'CCS Diagnosis Code', 'CCS Diagnosis Description', 'CCS Procedure Code',
       'CCS Procedure Description', 'Discharge Year',
       'Emergency Department Indicator', 'Ethnicity', 'Facility Id',
       'Facility Name', 'Gender', 'Health Service Area', 'Hospital County',
       'Length of Stay', 'Operating Certificate Number',
       'Operating Provider License Number', 'Other Provider License Number',
       'Patient Disposition', 'Payment Typology 1', 'Payment Typology 2',
       'Payment Typology 3', 'Race', 'Ratio of Total Costs to Total Charges',
       'Total Charges', 'Total Costs', 'Type of Admission',
       'Zip Code - 3 digits', 'yr'],


In [12]:
# saving the SparcsKnee to csv as a precaution 
destFol = os.getcwd()+'/dataFiles/nysHealth/'
filenames = os.listdir(destFol)

sparcsKnee.to_csv(os.getcwd()+'/dataFiles/nysHealth/sparcsKnee.csv')